In [1]:
import findspark
findspark.init('/home/yaniv/spark-3.0.0-preview2-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Lin_reg').getOrCreate()
df = spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [18]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler

In [7]:
ind = StringIndexer(inputCol='Cruise_line',outputCol='lineIdx')
#vec = OneHotEncoder(inputCol='lineIdx',outputCol='cruiseLineVec')
indexed = ind.fit(df).transform(df)
feat = VectorAssembler(inputCols=['lineIdx','Age','Tonnage',
            'passengers','length','cabins','passenger_density','crew']
                       ,outputCol='features')
output = feat.transform(indexed)

In [8]:
train,test = output.randomSplit([0.7,0.3])

In [9]:
lr = LinearRegression(labelCol='crew')
lr = lr.fit(train)
results = lr.evaluate(test)

In [6]:
results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-2.30926389122032...|
|-2.49800180540660...|
|7.105427357601002...|
|-2.66453525910037...|
|3.552713678800501...|
|1.776356839400250...|
|-1.59872115546022...|
|1.509903313490213...|
|                 0.0|
|-5.32907051820075...|
|8.881784197001252...|
|3.552713678800501...|
|-8.88178419700125...|
|-2.48689957516035...|
|-8.88178419700125...|
|-3.28626015289046...|
|-2.44249065417534...|
|5.329070518200751...|
|-4.44089209850062...|
|                 0.0|
+--------------------+
only showing top 20 rows



In [10]:
results.rootMeanSquaredError

1.1280548611355359e-15

In [11]:
results.meanAbsoluteError

6.449867095441385e-16

In [12]:
results.r2

1.0

In [13]:
unlabeled = test.select('features')

In [15]:
pred = lr.transform(unlabeled)

In [16]:
pred.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[8.0,29.0,45.0,11...|               5.2|
|[8.0,13.0,76.0,18...|               8.5|
|[1.0,26.0,47.262,...| 6.699999999999999|
|[1.0,11.0,110.0,2...|19.099999999999998|
|[6.0,11.0,91.0,20...|              9.99|
|[4.0,25.0,34.25,1...|               4.7|
|[4.0,11.0,90.0,22...|10.999999999999998|
|[1.0,22.0,70.367,...|               9.2|
|[2.0,6.0,113.0,37...|              12.0|
|[7.0,5.0,133.5,39...|             13.13|
|[1.0,23.0,70.367,...|               9.2|
|[1.0,19.0,70.367,...|               9.2|
|[1.0,6.0,110.2389...|11.499999999999998|
|[0.0,7.0,158.0,43...|              13.6|
|[9.0,21.0,19.093,...|               4.7|
|[2.0,15.0,108.806...|11.099999999999998|
|[1.0,28.0,46.052,...| 6.599999999999999|
|[16.0,6.0,30.2769...|3.5500000000000007|
|[7.0,10.0,58.825,...|               7.0|
|[18.0,15.0,83.338...| 9.450000000000001|
+--------------------+------------